# Generate Random NETLF Network

In [1]:
import numpy as np
import pandas as pd
import random as rn
import string
import matplotlib.pyplot as plt


In [2]:
N_groups = 9


## Custom Locations and Populations

In [14]:
x = np.array([0,500,1000])
y = np.array([0,500,1000])

xX,yY = np.meshgrid(x,y)

In [15]:
X = xX.flatten()
Y = yY.flatten()

In [13]:
X

array([[   0,  500, 1000],
       [   0,  500, 1000],
       [   0,  500, 1000]])

In [18]:
#X = rn.sample(range(0, 1000), N_groups) 
Y = rn.sample(range(0, 1000), N_groups)
names = list(string.ascii_lowercase)[:N_groups]
pops = rn.sample(range(100, 1000), N_groups)

In [20]:
pops = np.zeros(N_groups)+5000

In [22]:
data_nam = {"Group":names}
data_loc = {"Group":names, "X":X,"Y":Y}
data_pop = {"Group":names, "Population":pops}

In [23]:
frame_nam = pd.DataFrame(data_nam) 
frame_loc = pd.DataFrame(data_loc) 
frame_pop = pd.DataFrame(data_pop) 

In [24]:
frame_nam.to_csv("group_names.csv",index=False)
frame_loc.to_csv("group_locations.csv",index=False)
frame_pop.to_csv("group_populations.csv",index=False)

In [25]:
np.set_printoptions(suppress=True)
dist_direct = np.zeros((N_groups,N_groups))
dist_car = np.zeros((N_groups,N_groups))

In [26]:
for i in np.arange(N_groups):
    for j in np.arange(N_groups):
        dist_direct[i,j] = round(np.linalg.norm(np.array((X[i],Y[i]))-np.array((X[j],Y[j]))))
        dist_car[i,j] = round(np.abs(X[i]-X[j])+np.abs(Y[i]-Y[j]))
            

In [27]:
distance_cars = pd.DataFrame(dist_car, columns=names, index=names)
distance_direct = pd.DataFrame(dist_direct, columns=names, index=names)

In [29]:
distance_cars.to_csv("road_dist.csv")
distance_direct.to_csv("euc_dist.csv")

In [28]:
distance_cars

,a,b,c,d,e,f,g,h,i
a,0.0,513.0,1625.0,279.0,1090.0,1149.0,225.0,1058.0,1753.0
b,513.0,0.0,1112.0,766.0,577.0,662.0,712.0,545.0,1240.0
c,1625.0,1112.0,0.0,1346.0,535.0,774.0,1400.0,567.0,128.0
d,279.0,766.0,1346.0,0.0,811.0,1428.0,54.0,779.0,1474.0
e,1090.0,577.0,535.0,811.0,0.0,1239.0,865.0,32.0,663.0
f,1149.0,662.0,774.0,1428.0,1239.0,0.0,1374.0,1207.0,902.0
g,225.0,712.0,1400.0,54.0,865.0,1374.0,0.0,833.0,1528.0
h,1058.0,545.0,567.0,779.0,32.0,1207.0,833.0,0.0,695.0
i,1753.0,1240.0,128.0,1474.0,663.0,902.0,1528.0,695.0,0.0


## CSV Locations and Populations

In [2]:
vil = pd.read_csv("../data/villages_lat_lon.csv", delimiter=',') #village locations
vil_pops = np.genfromtxt("../data/village_pop_gender.dat", delimiter=',',skip_header=1) #village populations
pops = np.sum(vil_pops[:,1:],axis = 1)

In [3]:
sum(pops)

54359.0

In [4]:
N_groups = len(pops)

In [5]:
names = vil["VILLAGE"].values

In [6]:
X = vil["X_Centroid"].values - np.min(vil["X_Centroid"].values)
Y = vil["Y_Centroid"].values - np.min(vil["Y_Centroid"].values)

In [7]:
data_nam = {"Group":names}
data_loc = {"Group":names, "X":X,"Y":Y}
data_pop = {"Group":names, "Population":pops}

In [8]:
frame_nam = pd.DataFrame(data_nam) 
frame_loc = pd.DataFrame(data_loc) 
frame_pop = pd.DataFrame(data_pop) 

frame_nam.to_csv("group_names.csv",index=False)
frame_loc.to_csv("group_locations.csv",index=False)
frame_pop.to_csv("group_populations.csv",index=False)

In [9]:
np.set_printoptions(suppress=True)
dist_direct = np.zeros((N_groups,N_groups))
dist_car = np.zeros((N_groups,N_groups))

In [10]:
for i in np.arange(N_groups):
    for j in np.arange(N_groups):
        dist_direct[i,j] = round(np.linalg.norm(np.array((X[i],Y[i]))-np.array((X[j],Y[j]))))
        dist_car[i,j] = round(np.abs(X[i]-X[j])+np.abs(Y[i]-Y[j]))
            

In [13]:
dist_car

array([[    0.,  6932., 16146., ...,  8123.,  7636., 14279.],
       [ 6932.,     0., 23078., ...,  6477., 10060., 21211.],
       [16146., 23078.,     0., ..., 24269., 20047.,  3299.],
       ...,
       [ 8123.,  6477., 24269., ...,     0.,  4222., 22402.],
       [ 7636., 10060., 20047., ...,  4222.,     0., 18180.],
       [14279., 21211.,  3299., ..., 22402., 18180.,     0.]])

In [14]:
distance_cars = pd.DataFrame(dist_car, columns=names, index=names)
distance_direct = pd.DataFrame(dist_direct, columns=names, index=names)

distance_cars.to_csv("road_dist.csv")
distance_direct.to_csv("euc_dist.csv")

## Births and Deaths

For simplicity I assume that the population tree declines linearly with age. Therefore the proportion of the population in each age bracket can be modelled by a simple line- $y=mx+b$, where $x$ is the proportional size of each age bracket and $y$ is the bracket number (assuming $n$ brackets). As $x$ is a proportion we know:

$\sum_{i=1}^n x_i = 100$

We can then substitute $x$ and get: 

$\frac{\sum_{i=1}^ni - nb}{m} = 100$.

We can get:

$m= \frac{\sum_{i=1}^ni-n^2}{100-n x_n}$,

and 

$b = \frac{x_n\sum_{i=1}^ni -100n}{nx_n-100}$.

Then if we know the number of brackets and a single value for a bracket we can solve for all brackets. For example $n=16$ and $x_{16} =1$. Then $y=-1.423x+17.43$.

In [12]:
N_age_brackets = 16 #number of age brackets
W_brackets = 5 #width of age brackets (years)

In [38]:
xn = 1 #assuming 1% in 75+ pop
b = (xn*np.sum(np.arange(1,N_age_brackets+1))-100*N_age_brackets)/(xn*N_age_brackets-100)
m = (np.sum(np.arange(1,N_age_brackets+1))-N_age_brackets**2)/(100-N_age_brackets*xn)

In [39]:
prop = (np.arange(1,N_age_brackets+1)-b)/m/100

In [79]:
μd = 1/((np.arange(0,80,5)[::-1]+2.5)*365)


In [88]:
μb = np.zeros(16)
μb[3:9] =  0.925*sum(μd*prop)/sum(prop[3:9])

In [90]:
birth = {"Daily Rate":μb}
birth_frame = pd.DataFrame(birth)

death = {"Daily Rate":μd}
death_frame = pd.DataFrame(death)

age = {"Age Dist":prop}
age_frame = pd.DataFrame(age)


birth_frame.to_csv("birth_rates.csv",index=False)
death_frame.to_csv("mortality_rates.csv",index=False)
age_frame.to_csv("pop_age_dist.csv",index=False)